# 本地notebook设置代理

In [6]:
import os

os.environ["HTTP_PROXY"] = "socks5://127.0.0.1:10808"
os.environ["HTTPS_PROXY"] = "socks5://127.0.0.1:10808"
# 测试代理是否成功
import requests
try:
    text = requests.get("https://www.google.com").content
    print("连接成功")
except Exception as e:
    print("连接失败")

连接成功


# 流程
使用Google Earth Engine数据和Tensorflow进行深度学习的流程：
  1. 以TFRecord格式从Earth Engine导出训练/测试数据。
  2. 准备在TensorFlow模型中使用的数据。
  3. 在TensorFlow中训练和验证模型（Keras Sequential neural network）。
  4. 对从Earth Engine导出的TFRecord格式图像数据进行预测。
  5. 将分类的图像数据以TFRecord格式提取到Earth Engine。


# Earth Engine 授权、初始化

In [7]:
import ee

try:
    ee.Initialize()
    print("The Earth Engine package initialized successfully!")  
except Exception as e:
    ee.Authenticate()
    ee.Initialize() 
    print("The Earth Engine package initialized successfully!") 

Enter verification code:  4/1AY0e-g6yUBYw-2UAv9bfL0NKQ2RCXp53FW3Cc15rBsWirOl1rGB7qe--wAw


URLError: <urlopen error Remote end closed connection without response>

In [13]:
# 查看需要的库是否存在
import ee
import tensorflow as tf
import folium
import os
try:
  import google.colab
  import geemap.eefolium as geemap
except:
  import geemap

print(tf.__version__)

2.3.0


# 定义全局变量

In [8]:
# Earth Engine用户名，用来导入分类后的图像到Earth Engine的 assets 文件夹
USER_NAME = "wtzhu"

# 使用Landsat 8表面反射率数据
L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# 训练和测试的土地覆盖标签FeatureCollection
LABEL_DATA = ee.FeatureCollection('projects/google/demo_landcover_labels')

# 使用以下波段进行预测
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
LABEL = "landcover"
FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)
N_CLASSES = 3

# 训练集和测试集数据存储的名称
# 这里的训练集和测试集均为一些样本点，都有标签，利用训练集训练集训练模型，
# 测试集做精度评价
TRARN_FILE_PREFIX = "TrainData"
TEST_FILE_PREFIX = "TestData"
file_extension = ".tfrecord.gz"
# 存储到谷歌云盘中的路径
TRAIN_FILE_PATH = ""
TEST_FILE_PATH = ""
# 待预测图像路径
IMAGE_FILE_PREFIX = "Image_pixel_demo_"
OUTPUT_IMAGE_FILE = "Classified_pixel_demo.TFRecord"
EXPORT_REGION = ee.Geometry.Rectangle([-122.7, 37.3, -121.8, 38.00])
OUTPUT_ASSET_ID = os.path.join("users", USER_NAME, "Classified_pixel_demo")

# 数据准备和可视化

In [22]:
# 定义影像去云函数
def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudBitMask = ee.Number(2).pow(5).int()
    qa = image.select("pixel_qa")
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudBitMask).eq(0))
    return image.updateMask(mask).select()

# 选取2018年云覆盖中值复合输入数据
image = L8SR.filterDate("2018-01-01", "2018-12-31").map(maskL8sr).median()
# 地图可视化
map = geemap.Map(center=[40, -100], zoom=4)
map.setCenter(lon=38., lat=-122.5)
map.addLayer(ee_object=image, 
       vis_params={
           "bands": ["B4", "B3", "B2"], 
           "min": 0, 
           "max": 0.3
           },
       name="image"
       )
map.addLayerControl()
map